In [ ]:
#!pip install torch
import torch
print(torch.__version__)

1.12.1+cu113


Change torch.__version__ of following installation command.




In [ ]:
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 467 kB 31.9 MB/s 
     |████████████████████████████████| 7.9 MB 32.5 MB/s 
     |████████████████████████████████| 3.5 MB 67.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=7bfbff4fb427342edb779a6c617038ab55832a98c010c57e96c38060fc7435bd
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


Dataset

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import degree
import torch_geometric.transforms as T
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
dataset = TUDataset(root='/tmp/imdb', name='IMDB-BINARY')

### add code for using the node degree as input features.
if dataset.data.x is None:
  max_degree = 0
  degs = []
  for data in dataset:
      degs += [degree(data.edge_index[0], dtype=torch.long)]
      max_degree = max(max_degree, degs[-1].max().item())

  if max_degree < 1000:
      dataset.transform = T.OneHotDegree(max_degree)
  else:
      deg = torch.cat(degs, dim=0).to(torch.float)
      mean, std = deg.mean().item(), deg.std().item()
      dataset.transform = NormalizedDegree(mean, std)
### add train, val, test loader.
train_dataset, val_dataset, test_dataset = random_split(dataset,[800,100,100])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Extracting /tmp/imdb/IMDB-BINARY/IMDB-BINARY.zip
Processing...
Done!


In [ ]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

NameError: ignored

Model

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.conv3 = GCNConv(16, 16)
        self.lin1 = Linear(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        ### You can add more layers or alter the model structure. See geometric documents which layer or model you can use.
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        ### Pass aggregated representation to linear layer to make final prediction
        #x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        return F.log_softmax(x, dim=1)

Train function

In [ ]:
def train():
    model.train()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)

Validation function

In [ ]:
def val(loader):
    model.eval()
    loss_all = 0
    for data in loader:
        data = data.to(device)
        output = model(data)
        loss = F.nll_loss(output, data.y.view(-1))
        loss_all += loss.item() * data.num_graphs
    return loss_all / len(train_dataset)

Test function

In [ ]:
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


Main code

In [ ]:
def save_checkpoint(EPOCH, m, o, LOSS):
  torch.save({
            'epoch': EPOCH,
            'model_state_dict': m.state_dict(),
            'optimizer_state_dict': o.state_dict(),
            'loss': LOSS,
            }, "model_checkpoint.md")

In [ ]:
device = torch.device('cuda:0')

In [ ]:
model = GCN()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

number_of_epochs = 300 # You can change.

lowest_loss = float('inf')
for epoch in range(number_of_epochs):
    train_loss = train()
    #print("train_loss:",train_loss)
    val_loss = val(val_loader)
    print("val_loss:",val_loss)
    # Choose the lowest validation loss checkpoint (you can implement early stopping as well)
    if val_loss < lowest_loss:
      lowest_loss = val_loss
      save_checkpoint(epoch,model,optimizer,lowest_loss)
# Load the lowest validation loss checkpoint and check the performance.
checkpoint = torch.load("model_checkpoint.md")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
test_acc = test(test_loader)

In [ ]:
test_acc

0.7